<a href="https://colab.research.google.com/github/chrober24/Img2Price/blob/main/House2Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from PIL import Image  # or you can use the keras one to load images
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import numpy as np
import glob
import cv2
from sklearn.preprocessing import MinMaxScaler


def load_dataset(top_dir="/content/drive/MyDrive/Image_Classification/houses/"):
    images_dataset = []
    for root, dirs, files in os.walk(top_dir):
        for name in files:
            # print(os.path.join(root, name))
            img = np.array(Image.open(os.path.join(root, name)))
            img = img.astype('float32')            
            img /= 255

            images_dataset.append(img)
    return np.array(images_dataset)
img_data = load_dataset()
img_data[1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


array([[[0.01568628, 0.5176471 , 0.8117647 ],
        [0.01960784, 0.52156866, 0.8156863 ],
        [0.02352941, 0.5254902 , 0.81960785],
        ...,
        [0.36078432, 0.4       , 0.30588236],
        [0.18039216, 0.23137255, 0.12156863],
        [0.03137255, 0.08235294, 0.        ]],

       [[0.01568628, 0.5176471 , 0.8117647 ],
        [0.01960784, 0.52156866, 0.8156863 ],
        [0.02352941, 0.5254902 , 0.81960785],
        ...,
        [0.07450981, 0.11372549, 0.01960784],
        [0.11764706, 0.16862746, 0.06666667],
        [0.14509805, 0.19607843, 0.08627451]],

       [[0.01960784, 0.52156866, 0.8156863 ],
        [0.02352941, 0.5254902 , 0.81960785],
        [0.02745098, 0.5294118 , 0.8235294 ],
        ...,
        [0.06666667, 0.10196079, 0.01960784],
        [0.06666667, 0.11372549, 0.01960784],
        [0.01568628, 0.0627451 , 0.        ]],

       ...,

       [[0.6431373 , 0.6156863 , 0.5529412 ],
        [0.6431373 , 0.6156863 , 0.5529412 ],
        [0.6313726 , 0

In [ ]:
inputPath = "/content/drive/MyDrive/Image_Classification/houses/"
IMG_HEIGHT = 224
IMG_WIDTH = 224

def create_dataset(img_folder=inputPath):
   
    img_data_array=[]

    for file in os.listdir(img_folder):
       
        image_path= os.path.join(img_folder, file)
        image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
        image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
        image=np.array(image)
        image = image.astype('float32')
        image /= 255 
        img_data_array.append(image)
    return img_data_array

img_data = create_dataset()
img_data[0].shape

(224, 224, 3)

In [ ]:
cols = ["index" , "price"]
inputPath = "/content/housinginfo.csv"
df = pd.read_csv(inputPath, sep="," , header=None, names=cols)
df.head()

ID = df["index"]
price = df["price"].astype(str).astype(int)
price_norm = price / price.max()
price_norm


X = img_data
y = price_norm


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X), len(X_train), len(X_test), len(y), len(y_train), len(y_test)

(600, 480, 120, 600, 480, 120)

In [ ]:
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from pyimagesearch import datasets
from pyimagesearch import models
import numpy as np
import argparse
import locale
import os

def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
	# initialize the input shape and channel dimension, assuming
	# TensorFlow/channels-last ordering
	inputShape = (height, width, depth)
	chanDim = -1

  	# define the model input
	inputs = Input(shape=inputShape)
	# loop over the number of filters
	for (i, f) in enumerate(filters):
		# if this is the first CONV layer then set the input
		# appropriately
		if i == 0:
			x = inputs
		# CONV => RELU => BN => POOL
	  x = Conv2D(f, (3, 3), padding="same")(x)
	  x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
  
  	# flatten the volume, then FC => RELU => BN => DROPOUT
	  x = Flatten()(x)
  	x = Dense(16)(x)
	  x = Activation("relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
	  x = Dropout(0.5)(x)
	# apply another FC layer, this one to match the number of nodes
	# coming out of the MLP
  	x = Dense(4)(x)
	  x = Activation("relu")(x)
	# check to see if the regression node should be added
	  if regress:
		  x = Dense(1, activation="linear")(x)
	# construct the CNN
  	  model = Model(inputs, x)
	# return the CNN
	return model

model = models.create_cnn(224, 224, 3, regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(x=X_train, y=y_train, 
    validation_data=(X_test, y_test),
    epochs=200, batch_size=8)

IndentationError: ignored